In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import matplotlib as mpl
from mpltools import annotation
import time

from matplotlib import rc
import seaborn as sns
custom_params = {
    "xtick.direction": "in",
    "ytick.direction": "in",
    "lines.markeredgecolor": "k",
    "lines.markeredgewidth": 0.3,
    "figure.dpi": 200,
    "text.usetex": True,
    "font.family": "serif",
}

sns.set_theme(context = "notebook", style="ticks", rc=custom_params)

BlueUB = (0, 157/255, 224/255)
BrownUB = (68/255, 58/255, 49/255)

In [5]:
from DoubleWalls_Overdamped_Langevin import DoubleWallsLangevin

a = 1.519e-6
H = 20e-6
lD = 88.0e-9
lB = 526e-9
B = 5.0
T = 300
eta0 = 0.001

dt = 0.01
Nt = 500_000

tc_0 = time.time() 
brown = DoubleWallsLangevin(dt=dt, Nt=Nt,
                                               a=a, H=H, lD=lD, lB=lB, B=B,
                                               Nt_sub=1, eta0=eta0,
                                               T=T, R0=None)#Instance of DoubleWallsLangevin class
brown.trajectory() #Compute trajectory of "brown"
print("Compute time= ", time.time()-tc_0)

Compute time=  735.1434528827667


## Plot diffusion coefficients 

In [ ]:
plt.figure(figsize=(1.5 * 3.375, 1.5 * 3.375 / 1.68), tight_layout=True)

epsilon =  brown.H*1e-8
z_1 = np.linspace(- brown.H+epsilon, - brown.H+1000e-6, 100000)
z_2 = np.linspace(- (brown.H-epsilon),  brown.H-epsilon, 1000)
D0 = brown.D0

## Diffusion à un mur
ax1 = plt.subplot(121)
ax1.loglog((z_1+ brown.H)/ brown.a,  brown.D_x(z_1)/D0, color=BlueUB)
ax1.plot((z_1+ brown.H)/ brown.a, brown.D_z(z_1)/D0, color="tab:green")
ax1.set(
    xlabel = r"$(z_t+H)/a$",
    ylabel = r"$D_i/D_0$",
    xlim = (1e-4, 2e1),
    ylim = (1e-4, 2e0),
)
ax1.text(0.06, 0.92, r"(a)", transform=ax1.transAxes)
locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
ax1.yaxis.set_major_locator(locmaj)
ax1.yaxis.set_minor_locator(locmin)
ax1.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())

ax1.xaxis.set_major_locator(locmaj)
ax1.xaxis.set_minor_locator(locmin)
ax1.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())

ax1_ticks = ax1.get_yticklabels()
for i in ax1_ticks:
    i.set_visible(False)
for n, i in enumerate(ax1_ticks):
    if n%1==0:i.set_visible(True)
ax1_ticks = ax1.get_xticklabels()
for i in ax1_ticks:
    i.set_visible(False)
for n, i in enumerate(ax1_ticks):
    if n%2==0:i.set_visible(True)
    

## Diffusion à deux mur
ax2 = plt.subplot(122)
ax2.plot(z_2/brown.a, brown.D_x(z_2)/D0, color=BlueUB)
ax2.plot(z_2/brown.a, brown.D_z(z_2)/D0, color="tab:green")
ax2.set(
    xlabel = r"$z_t/a$",
)
ax2.text(0.06, 0.92, r"(b)", transform=ax2.transAxes)


# fig.tight_layout()
# plt.savefig("Diffusion.pdf")

## Shoot of initial altitude $Z_0$ over $P_\mathrm{eq}$

In [ ]:
u = np.linspace(0.01,0.99, 100) # u in ]0,1[
F_inverse = brown.sample() # Call function

In [ ]:
# Shoot 1000 times z_eq value over Peq
z0_eq = brown.return_samples(1000) # Compute z_eq from P_eq(z)

bins=100
binsPositions_eq, _, hist_Z0eq = brown.logarithmic_hist(position=z0_eq+brown.H, begin=1e-15, stop=7e-6, num=bins, )
P_Z0eq = hist_Z0eq / np.trapz(hist_Z0eq, binsPositions_eq*1e6)

In [ ]:
# Theoritical Peq
epsilon = brown.H*1e-3
z0_Peq = np.logspace(np.log(1e-20)/np.log(2), np.log(8e-6)/np.log(2), 1000, base=2)

Peq_theo =  brown.P_eq_Z(z0_Peq-brown.H)

In [ ]:
plt.figure(figsize=(1.5 * 3.375, 1.5 * 3.375 / 1.68), tight_layout=True)
ax1 = plt.subplot(121)
ax1.plot(u, F_inverse(u), color=BlueUB)
ax1.set(
    xlabel = r"$U$",
    ylabel = r"$F^{-1}_\mathrm{eq}(U)$",
)
ax1.text(0.06, 0.92, r"(a)", transform=ax1.transAxes)

ax2 = plt.subplot(122)
ax2.semilogy((binsPositions_eq)*1e6, P_Z0eq, "o", color=BlueUB)
ax2.semilogy( z0_Peq*1e6, Peq_theo/np.trapz( Peq_theo , z0_Peq*1e6), "k-",)
ax2.set(
    xlabel = r"$Z_0~\mathrm{(\mu m)}$",
    ylabel = r"$P\mathrm{eq}(Z_0)$",
    xlim = (-0.5, 5),
    ylim = (5e-4, 5e0)
)
ax2.text(0.06, 0.92, r"(b)", transform=ax2.transAxes)

# plt.savefig("SampleFunction.pdf")

## Trajectory

In [ ]:
plt.figure(figsize=(1.5 * 3.375, 1.5 * 3.375 / 1.68), tight_layout=True)
M=10000
## TRAJECTOIRE
ax1 = plt.subplot(111)
ax11=ax1.twinx()
l1 = ax11.plot(np.arange(Nt)[:M] * dt, (brown.Zn[:M]+H)*1e6, color="tab:green", label=r"$Z_n+H$")
ax11.set_ylabel("$Z_n~\mathrm{(\mu m)}$",color="green")
ax11.tick_params(axis='y', labelcolor="green")
ax1.tick_params(axis='y', labelcolor="tab:blue")

# ax11.set_ylim(0, 4)
l2 = ax1.plot(np.arange(Nt)[:M] * dt, brown.Xn[:M]*1e6, color=BlueUB, label=r"$X_n$")
lns = l1 + l2
labs = [l.get_label() for l in lns]
# ax1.legend(lns, labs, loc='upper left', frameon=False, fontsize=8)
ax1.set(
    xlabel = r"$t_n~\mathrm{(s)}$",
    ylabel = r"$X_n ~\mathrm{(\mu m)}$",
)
ax1.yaxis.label.set_color(BlueUB)

# plt.savefig("Trajectory.pdf")

## Standard deviation

In [ ]:
tau_x, msd_x = brown.MSD(axis="x") #Invocation de la méthode MSD pour calculer la déviation standard de la trajectoire
tau_z, msd_z = brown.MSD(axis="z")
dt_theo = np.linspace(brown.dt*1e-1, brown.dt*brown.Nt*1e1, 100)

## Equilibrium distribution

In [ ]:
# Numerical datas
bins=100
binsPositions, _, hist_z = brown.logarithmic_hist(position=brown.Zn+brown.H, begin=1e-15, stop=7e-6, num=bins, )
pdf = hist_z / np.trapz(hist_z, binsPositions*1e6)

In [ ]:
# Theoritical datas
z_Peq = np.logspace(np.log(1e-20)/np.log(2), np.log(8e-6)/np.log(2), 1000, base=2)
epsilon = brown.H*1e-3
z_theo = np.linspace(-brown.H-epsilon, brown.H-epsilon, 1000)

Peq_normalised = brown.P_eq_Z(z_theo)/np.trapz( brown.P_eq_Z(z_theo) , z_theo)

mean_Dx = np.trapz(Peq_normalised*brown.D_x(z_theo), z_theo)
mean_Dz = np.trapz(Peq_normalised*brown.D_z(z_theo), z_theo)

dz_theo = np.linspace(-2*(brown.H-epsilon), 2*(brown.H-epsilon), 1000)
P_dZ_longTime = [np.trapz(brown.P_eq_Z(z_theo)*brown.P_eq_Z(z_theo+dz), z_theo) for dz in dz_theo] #Compute P(dZ) at long time
P_dZ_longTime = P_dZ_longTime/np.trapz(P_dZ_longTime, dz_theo)
plateauZ = np.trapz(dz_theo**2 * P_dZ_longTime, dz_theo)

## Plots

In [ ]:
plt.figure(figsize=(1.5 * 3.375, 1.5 * 3.375 / 1.68), tight_layout=True)

ax2 = plt.subplot(121)
ax2.semilogy(binsPositions*1e6, pdf, "d", color="green", label=r"$\mathrm{Simulation}$")
ax2.plot( z_Peq*1e6, brown.P_eq_Z(z_Peq-brown.H)/np.trapz( brown.P_eq_Z(z_Peq-brown.H) , z_Peq*1e6), "k-", label=r"$\mathrm{Theory}$",)
ax2.set(
    xlabel = r"$Z_{n}+H~\mathrm{(\mu m)}$", 
    ylabel = r"$P_\mathrm{eq}(Z_n)$",
    xlim = (None,4.5),
    ylim = (1e-3,None),
)
ax2.text(0.85, 0.92, r"(a)", transform=ax2.transAxes)


ax1 = plt.subplot(122)
ax1.loglog(tau_x, msd_x*1e12, "o", color=BlueUB)
ax1.loglog(tau_z, msd_z*1e12, "d", color="tab:green")
ax1.plot(dt_theo, 2*mean_Dx*1e12*dt_theo, "--", color="k")#mediumblue
ax1.plot(dt_theo, 2*mean_Dz*1e12*dt_theo, "-", color="k") #darkgreen
ax1.plot(np.linspace(10, 10000, 100), plateauZ*1e12*np.ones(100), ":", color="k")
ax1.set(
    xlabel = r"$\tau ~\mathrm{(s)}$",
    ylabel = r"$\langle (R^{(i)}_n)^2 \rangle~\mathrm{(\mu m)}^2$",
    xlim=(brown.dt/2, brown.dt*brown.Nt),
    ylim=(1e-16*1e12, 2e-10*1e12),
)
ax1.text(0.05, 0.92, r"(b)", transform=ax1.transAxes)
locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
ax1.yaxis.set_major_locator(locmaj)
ax1.yaxis.set_minor_locator(locmin)
ax1.yaxis.set_minor_formatter(mpl.ticker.NullFormatter())

locmaj = mpl.ticker.LogLocator(base=10.0, subs=(1.0, ), numticks=100)
locmin = mpl.ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * .1, numticks=100)
ax1.xaxis.set_major_locator(locmaj)
ax1.xaxis.set_minor_locator(locmin)
ax1.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())
ax1_ticks = ax1.get_xticklabels()
for i in ax1_ticks:
    i.set_visible(False)
for n, i in enumerate(ax1_ticks):
    if n%2==0:i.set_visible(True)

# plt.savefig("Analyse.pdf")